#### 1. 利用Logistic回归预测疝气病症的病马的死亡率

- 数据集: [下载地址](http://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/)
- 数据集描述 :
- horse-colic.data : 训练数据
- horse-colic.names : 数据集字段描述
- horse-colic.test : 测试数据
- 补充 : 数据集中有 30% 缺失值

#### 1.1 数据读取

- 对缺失的数据使用当前列的均值进行处理
- 对数据作归一化处理

- 数据字段详解 :
    1. surgery
        - 1 : 做过手术
        - 2 : 未做过手术
    2. Age (年龄)
        - 1 : 成年马
        - 2 : 未成年马
    3. Hospital Number (医院编号)
    4. rectal temperature (直肠温度)
    5. pulse (脉搏)
        - 每分钟心跳数
    6. respiratory rate (呼吸率)
    7. temperature of extremities (是指温度)
        - 1 = Normal
        - 2 = Warm
        - 3 = Cool
        - 4 = Cold

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# 设置显示的中文字题为宋体
font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=20)

In [4]:
train_path = './csv_data/horse-colic-train.csv'
test_path = './csv_data/horse-colic-test.csv'

column = []

train_origin_data = pd.read_csv(train_path, header=None)
test_origin_data = pd.read_csv(test_path, header=None)

print(train_origin_data.head())
print(test_origin_data.head())

                                                   0
0  2 1 530101 38.50 66 28 3 3 ? 2 5 4 4 ? ? ? 3 5...
1  1 1 534817 39.2 88 20 ? ? 4 1 3 4 2 ? ? ? 4 2 ...
2  2 1 530334 38.30 40 24 1 1 3 1 3 3 1 ? ? ? 1 1...
3  1 9 5290409 39.10 164 84 4 1 6 2 2 4 4 1 2 5.0...
4  2 1 530255 37.30 104 35 ? ? 6 2 ? ? ? ? ? ? ? ...
                                                   0
0  2 1 528626 38.50 54 20 ? 1 2 2 3 4 1 2 2 5.90 ...
1  2 1 527950 37.60 48 36 ? ? 1 1 ? 3 ? ? ? ? ? ?...
2  1 1 535263 37.7 44 28 ? 4 3 2 5 4 4 1 1 ? 3 5 ...
3  1 1 534523 37 56 24 3 1 4 2 4 4 3 1 1 ? ? ? 35...
4  2 1 528926 38.00 42 12 3 ? 3 1 1 ? 1 ? ? ? ? 2...


In [5]:
train_origin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       300 non-null    object
dtypes: object(1)
memory usage: 2.5+ KB


In [11]:
test_origin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       68 non-null     object
dtypes: object(1)
memory usage: 672.0+ bytes


- 通过观察上述输出结果可以看到很多的 ? , 这些都是错误数据, 处理方式有以下几种 :
    1. 直接删除带 ? 的数据的对应行
    2. 使用 ? 对应列的均值替换

#### 1.2 数据预处理

- 该样本主要的问题的是有 30% 的缺失值(来源于 horse-colic.names 描述), 缺失值数据显示为 ?
    1. 使用可用的特征的均值来填补缺失值
    2. 使用特殊值来填补缺失值
    3. 忽略有缺失值的样本
    4. 使用相似样本的均值填补缺失值
    5. 使用另外的机器学习算法预测缺失值

- 处理缺失值的步骤 :
    1. 对 **特征** 缺失值先使用 0 值替换
    2. 计算每一列的均值, 然后使用均值替换掉缺失值
    3. 对 **类别** 缺失值, 删除缺失的行

In [19]:
train_origin_data[]

,0
1,1 1 534817 39.2 88 20 ? ? 4 1 3 4 2 ? ? ? 4 2 ...
2,2 1 530334 38.30 40 24 1 1 3 1 3 3 1 ? ? ? 1 1...
